<a href="https://colab.research.google.com/github/JuanPoG/Tareas_Multimedia/blob/main/Proyecto_2/Proyecto_2_PobleteJuan_20521642_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>

</center></h1>
<font size="6"><center>
EIE 401
PROCESAMIENTO DIGITAL MULTIMEDIA
</center></font>


<center><h2>Proyecto 2</h2></center>
<center><h3>Procesamiento digital de señales</h3></center>
<center><h3>Profesor: Jorge Cardenas</h3></center>

<center><h3>Por: Juan Pablo Poblete Galvez</h3></center>
<center><h5>Pontificia Universidad Catolica de Valparaiso</h5></center>


In [ ]:
from IPython.display import IFrame, display
filepath = "Proyecto_2_EIE401.pdf"
IFrame(filepath, width=700, height=400)

En la dirección https://huggingface.co/datasets/astroboy1/metasurfaces_V1_30-90GHZ encontrarás el archivo batch_may2024.zip
 que contiene las imagenes que usarás para el procesamiento.

## 1. Carga de Datos


- En los archivos, encontrarás un folder con imagenes de 3 categorías: cross, box, circle.
Cada imagen tiene una medida de 512x512 pixeles y 3 canales. Se han acentuado los azules y rojos para facilitar el proceso de obtención de características.

- Puedes extraer la clase a la que pertenece la imagen, desde el nombre del archivo. Ahí mismo encuentras números de serie que te permiten identificar de forma específica una imagen.

- Se te entrega 3 imagenes de referencia de 16x16 pixeles, las que debes usar como plantilla para el proceso de clasificación de la imagen.


In [1]:
!git clone https://github.com/Jorgecardenas1/EIE_401_MULTIMEDIA.git

Cloning into 'EIE_401_MULTIMEDIA'...
remote: Enumerating objects: 347, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 347 (delta 1), reused 1 (delta 0), pack-reused 341
Receiving objects: 100% (347/347), 44.47 MiB | 15.20 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [2]:
import zipfile

ruta_zip = '/content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/imagenes.zip'

with zipfile.ZipFile(ruta_zip,'r') as f:
  f.extractall('imagenes')

ruta_imagenes='/content/imagenes/processed512'

## 2. Transformar imagen
Recuerda que la tarea es lograr el procesamiento más rápido posible.
Para lograr esto, realiza las operaciones que consideres necesarias sobre la imagen original para el posterior trabajo de Matching con las plantillas dadas.

In [3]:
import os
import cv2
import numpy as np
import pandas as pd

#direcciones
input_dir = '/content/imagenes/processed512'
output_dir_transformed = '/content/imagenes'
output_dir_convolved = '/content/imagenes/salida_convolved'
template_dir = '/content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/'
result_file = 'resultados.csv'

#crea las rutas
if not os.path.exists(output_dir_transformed):
    os.makedirs(output_dir_transformed)

if not os.path.exists(output_dir_convolved):
    os.makedirs(output_dir_convolved)

In [7]:
def transform_image(image_path, output_path):
    """
    escala las imagenes a un tamaño menor para
    hacer mas eficiente el proceso de matching
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return

    #escala imagenes
    img_resized = cv2.resize(img, (256, 256))
    cv2.imwrite(output_path, img_resized)

#procesa las imagenes
for filename in os.listdir(input_dir):
    """
    para aplicar a todas las imagenes se utiliza un ciclo for
    para que tome los archivos que sean .jpg o .png
    """
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir_transformed, filename)
        transform_image(img_path, output_path)

## 3. Operaciones de convolución
Para extraer las caracteristicas, debes realizar tantas operaciones de convolución como creas necesarias.
El resultado de tus convoluciones vas a usarlo para el proceso de matching.


In [8]:
def apply_convolution(image):
    """
    aplica kernels para extraer caracteristicas
    de las imagenes para facilitar el proceso de matching
    """
    kernel_sharpen = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    kernel_edge = np.array([[-1,-1,-1], [0,0,0], [1,1,1]])

    sharpened = cv2.filter2D(image, -1, kernel_sharpen)
    edged = cv2.filter2D(image, -1, kernel_edge)

    combined = cv2.addWeighted(sharpened, 0.5, edged, 0.5, 0)

    return combined

def preprocess_image(image_path, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return

    convolved_img = apply_convolution(img)
    edges = cv2.Canny(convolved_img, 50, 150)

    cv2.imwrite(output_path, edges)

In [9]:
#aplica las convoluciones
for filename in os.listdir(output_dir_transformed):
    """
    para aplicar a todas las imagenes se utiliza un ciclo for
    para que tome los archivos que sean .jpg o .png
    """
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(output_dir_transformed, filename)
        output_path = os.path.join(output_dir_convolved, filename)
        preprocess_image(img_path, output_path)

## 4. Matching
El proceso de matching debe considerar que vas a usar las imagenes de referencia para calcular la correlación
![image.png](attachment:6850d57c-7e6f-4acf-ac3b-d9a8feb071cb.png)

In [22]:
from timeit import default_timer as timer

# Lee las plantillas y aplica convolución
templates = {
    'cross': cv2.imread(os.path.join(template_dir, 'cross_templae.png'), cv2.IMREAD_GRAYSCALE),
    'box': cv2.imread(os.path.join(template_dir, 'box_templae.png'), cv2.IMREAD_GRAYSCALE),
    'circle': cv2.imread(os.path.join(template_dir, 'circ_templae.png'), cv2.IMREAD_GRAYSCALE)
}

processed_templates = {category: apply_convolution(template) for category, template in templates.items()}

def match_template(image, template):
    result = cv2.matchTemplate(image, template, cv2.TM_SQDIFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    return max_val

def perform_matching(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None

    preprocessed_img = apply_convolution(img)

    #realiza matching con cada plantilla
    match_scores = {category: match_template(preprocessed_img, template) for category, template in processed_templates.items()}
    best_match = max(match_scores, key=match_scores.get)
    best_score = match_scores[best_match]

    return best_match, best_score

start = timer()

#procesa cada imagen
results = []
for filename in os.listdir(output_dir_convolved):
    """
    para aplicar a todas las imagenes se utiliza un ciclo for
    para que tome los archivos que sean .jpg o .png
    """
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(output_dir_convolved, filename)
        best_match, best_score = perform_matching(img_path)
        results.append((filename, best_match, best_score))

end = timer()
print(end - start)


79.85968539100008


# 5. Clasificación
De cada operación realizada obtendrás valores para inferir si la imagen bajo estudio corresponde con alguna de las 3 categorías: cross, box, circle.

Debes almacenar cada resultado obtenido para cada imagen, hallar el error de clasificación y generar un informe del error general al clasificar todas las imagenes.



Presenta aquí el resultado de clasificación numérico, así como  dos ejemplos verdaderos y dos ejemplos falsos de imagenes clasificadas.


In [23]:
def get_real_category(filename):
    if 'cross' in filename.lower():
        return 'cross'
    elif 'box' in filename.lower():
        return 'box'
    elif 'circle' in filename.lower():
        return 'circle'
    else:
        return None

#añade la categoría real a los resultados
resultados = []
for filename, predicted_category, best_score in results:
    real_category = get_real_category(filename)
    resultados.append({
        'filename': filename,
        'predicted_category': predicted_category,
        'best_score': best_score,
        'real_category': real_category
    })

#guarda los resultados en archivo CSV
df_results = pd.DataFrame(resultados)
df_results.to_csv(result_file, index=False)

#evaluación error de clasificación
df_results['correct'] = df_results['predicted_category'] == df_results['real_category']

#calcula la precision
accuracy = df_results['correct'].mean()
classification_error = 1 - accuracy

#reporte
report = f"""
Clasificación:
-------------------------
Total de imágenes: {len(df_results)}
Clasificaciones correctas: {df_results['correct'].sum()}
Clasificaciones incorrectas: {len(df_results) - df_results['correct'].sum()}
Precisión: {accuracy * 100:.2f}%
Error de Clasificación: {classification_error * 100:.2f}%
"""

with open('informe.txt', 'w') as f:
    f.write(report)

print(report)


Clasificación:
-------------------------
Total de imágenes: 14328
Clasificaciones correctas: 5923
Clasificaciones incorrectas: 8405
Precisión: 41.34%
Error de Clasificación: 58.66%



# 6. Medición
Recuerda que debes imprimir el tiempo que tarda tu algoritmo en procesar todas las imagenes en la base de datos. Usa el siguiente fragmento de código, ubicando el inicio y fin donde corresponda.





In [ ]:
from timeit import default_timer as timer

start = timer()

print(23*2.3)

end = timer()
print(end - start)

